# Importing Dependencies

In [1]:
import numpy as np
import os

import cv2
import matplotlib.pylab as plt

import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# Image Enhancement Dataset

In [3]:
data_dir = 'imgdata_v2/orginal_dataset/'

## Sharpen Function

In [4]:
kernel_sharpen = np.array([[-1,-1,-1],
                          [-1,9,-1],
                          [-1,-1,-1]])
def sharpen(img):
    img_sharpen = cv2.filter2D(img, -1, kernel_sharpen)
    return img_sharpen

## Crop Function

In [5]:
lower = np.array([0,0,0])
higher = np.array([30,30,30])

def crop(img):
    img_copy = img.copy()
    mask = cv2.inRange(img_copy, lower, higher)
    inv_mask = cv2.bitwise_not(mask)

    cont, _ = cv2.findContours(inv_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(img_copy, cont, -1, (0,255,0), 20)

    c = max(cont, key = cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    img_cropped = img[y:y+h,x:x+w]
    return img_cropped

# Preprocessing Image

In [6]:
for img_class in os.listdir(data_dir):
    for img in os.listdir(os.path.join(data_dir, img_class)):
        img_path = os.path.join(data_dir, img_class, img)
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

        if img[0][0][0] == 255:
            os.remove(img_path)
            print('{} White-background image successfully removed'.format(img_path))
        else:
            img_crop = crop(img)
            img_resize = cv2.resize(img_crop, (512, 512))
            img_sharpen = sharpen(img_resize)
            plt.imsave(img_path, img_sharpen)    

imgdata_v2/orginal_dataset/cataract\2174_right.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2175_left.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2176_left.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2177_right.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2178_right.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2179_right.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2180_left.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2180_right.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2181_left.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2181_right.jpg White-background image successfully removed
imgdata_v2/orginal_dataset/cataract\2182_left.jpg Whit

# Splitting Dataset 

In [7]:
for img_class in os.listdir(data_dir):
    print(img_class,': ', len(os.listdir(os.path.join(data_dir, img_class))))

cataract :  1009
normal :  1072


In [8]:
import splitfolders

splitfolders.ratio(data_dir, 
                   output='imgdata_v2/split_dataset/',
                   seed=26,
                   ratio=(.7, .2, .1),
                   group_prefix=None)

Copying files: 2081 files [00:02, 831.57 files/s]


In [9]:
split_path = 'imgdata_v2/split_dataset/'

for split in os.listdir(split_path):
    for img_class in os.listdir(os.path.join(split_path, split)):
        print(img_class, split, ': ', len(os.listdir(os.path.join(split_path, split, img_class))))
    print()

cataract test :  102
normal test :  108

cataract train :  706
normal train :  750

cataract val :  201
normal val :  214



In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img, img_to_array

In [11]:
train_dir = os.path.join(split_path, 'train')

In [12]:
try:
    os.mkdir(os.path.join(split_path, 'train_aug'))
except Exception as e:
    print('Folder already exists.')

train_aug_dir = os.path.join(split_path, 'train_aug')

In [13]:
train_datagen = ImageDataGenerator(brightness_range=(0.1, 1.5),
                             horizontal_flip=True,
                             vertical_flip=True,
                             fill_mode='constant')

for img_class in os.listdir(train_dir):
    aug_class_dir = os.path.join(train_aug_dir, img_class)
    try:
        os.mkdir(aug_class_dir)
    except Exception as e:
        print(img_class, 'Folder already exists')
        
    for img in os.listdir(os.path.join(train_dir, img_class)):
        img = load_img(os.path.join(train_dir, img_class, img))
        img = img_to_array(img)
        img = img.reshape((1, )+img.shape)
        i=0
        for batch in train_datagen.flow(img,
                                       batch_size=16,
                                       save_to_dir=aug_class_dir,
                                       save_format='jpg',
                                       save_prefix='aug'):
            i += 1
            if i>5:
                break;
    print(img_class, 'augmented : ', len(os.listdir(aug_class_dir)))

cataract augmented :  3447
normal augmented :  3604
